### __Importing the dependencies__

In [3]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE
from utils import hybrid_method

### __Loading the dataset__

In [4]:
data = load_breast_cancer()
data = pd.DataFrame(np.c_[data['data'], data['target']],
                  columns= np.append(data['feature_names'], ['target']))

In [5]:
data.head(5)

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0.0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0.0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0.0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0.0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0.0


In [6]:
X_train, X_test, y_train, y_test = train_test_split(data.drop(labels=['target'], axis=1), 
                                                    data.target, test_size=0.2,
                                                    random_state=0)
X_train.shape, X_test.shape

((455, 30), (114, 30))

### __Recursive Feature Elimination With Random Forests Importance__

#### __Example 1__

<div align="justify">

This method is slightly **different from the guide**, as it use a different stopping criterion: the desired number of features to select is eventually reached.

</div>

In [7]:
sel_ = RFE(RandomForestClassifier(n_estimators=20), n_features_to_select=10)
sel_.fit(X_train.fillna(0), y_train)

,estimator,RandomForestC...estimators=20)
,n_features_to_select,10
,step,1
,verbose,0
,importance_getter,'auto'
,n_estimators,20
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0


In [8]:
selected_feat = X_train.columns[(sel_.get_support())]

In [9]:
print(selected_feat)

Index(['mean perimeter', 'mean concave points', 'area error', 'worst radius',
       'worst texture', 'worst perimeter', 'worst area', 'worst smoothness',
       'worst concave points', 'worst symmetry'],
      dtype='object')


#### __Example 2__

<div align="justify">

Recursive feature elimination with RandomForest
with the method same as the guide
1. Rank the features according to their importance derived from a machine learning algorithm: it can be tree importance, or LASSO / Ridge,  or the linear / logistic regression coefficients.
2. Remove one feature -the least important- and build a machine learning algorithm utilizing the remaining features.
3. Calculate a performance metric of your choice: roc-auc, mse, rmse, accuracy.
4. If the metric decreases by more of an arbitrarily set threshold, then that feature is important and should be kept. Otherwise, we can remove that feature.
5. Repeat steps 2-4 until all features have been removed (and therefore evaluated) and the drop in performance assessed.

</div>

In [11]:
features_to_keep = hybrid_method.recursive_feature_elimination_rf(X_train=X_train,
                                                           y_train=y_train,
                                                           X_test=X_test,
                                                           y_test=y_test,
                                                           tol=0.001)


testing feature:  mean radius  which is feature  1  out of  30
New Test ROC AUC=0.9941251190854239
All features Test ROC AUC=0.9968243886948238
Drop in ROC AUC=0.0026992696093999236
keep:  mean radius

testing feature:  mean texture  which is feature  2  out of  30
New Test ROC AUC=0.9936487773896475
All features Test ROC AUC=0.9968243886948238
Drop in ROC AUC=0.0031756113051762958
keep:  mean texture

testing feature:  mean perimeter  which is feature  3  out of  30
New Test ROC AUC=0.9968243886948238
All features Test ROC AUC=0.9968243886948238
Drop in ROC AUC=0.0
remove:  mean perimeter

testing feature:  mean area  which is feature  4  out of  30
New Test ROC AUC=0.9960304858685297
All features Test ROC AUC=0.9968243886948238
Drop in ROC AUC=0.0007939028262941017
remove:  mean area

testing feature:  mean smoothness  which is feature  5  out of  30
New Test ROC AUC=0.9965068275643061
All features Test ROC AUC=0.9960304858685297
Drop in ROC AUC=-0.0004763416957763722
remove:  mean 

In [12]:
features_to_keep

['mean radius',
 'mean texture',
 'mean compactness',
 'mean symmetry',
 'mean fractal dimension',
 'radius error',
 'texture error',
 'perimeter error',
 'area error',
 'smoothness error',
 'compactness error',
 'concavity error',
 'concave points error',
 'fractal dimension error',
 'worst radius',
 'worst area',
 'worst smoothness',
 'worst concave points']